In [4]:
import os
import sys
sys.path.append('../va_pipeline/')

In [6]:
import subprocess
import pandas as pd
import cv2
import numpy as np

from sensor import *


In [8]:
os.path.exists('./model_tests/')

False